In [ ]:
#default_exp parser
%load_ext autoreload
%autoreload 2

# Parser
> Funções principais de Leitura e Processamento dos dados provenientes dos Bancos de Dados

In [ ]:
#export
from pathlib import Path
import re
import xml.etree.ElementTree as et
from zipfile import ZipFile
from datetime import datetime
import collections
import pandas as pd
import pandas_read_xml as pdx
from anateldb.query import *

In [ ]:
#export
def read_stel(pasta, update=False):
    """Lê o banco de dados salvo localmente do STEL. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01."""
    if update:
        update_stel(pasta)
    if (file := Path(f'{pasta}/stel.fth')).exists():
        stel = pd.read_feather(file)
    elif (file := Path(f'{pasta}/stel.csv')).exists():
        stel = pd.read_csv(file)
    elif (file := Path(f'{pasta}/Base_de_Dados.xlsx')).exists():
        stel = pd.read_excel(file, sheet_name='Stel', engine='openpyxl')
    else:
        update_stel(pasta)
        try:
            stel = pd.read_feather(Path(f'{pasta}/stel.fth'))
        except FileNotFoundError as e:
            raise ConnectionError("Base de Dados do Stel inexistente e não foi possível atualizá-la" ) from e
    return stel

In [ ]:
#export
def read_radcom(pasta, update=False):
    """Lê o banco de dados salvo localmente de RADCOM. Opcionalmente o atualiza pelo Banco de Dados ANATELBDRO01."""
    if update:
        update_radcom(pasta)
    if (file := Path(f'{pasta}/radcom.fth')).exists():
        radcom = pd.read_feather(file)
    elif (file := Path(f'{pasta}/radcom.csv')).exists():
        radcom = pd.read_csv(file)
    elif (file := Path(f'{pasta}/Base_de_Dados.xlsx')).exists():
         radcom = pd.read_excel(file, sheet_name='Radcom', engine='openpyxl')
    else:
        update_radcom(pasta)
        try:
            radcom = pd.read_feather(Path(f'{pasta}/radcom.fth'))
        except FileNotFoundError as e:
            raise ConnectionError("Base de Dados do Stel inexistente e não foi possível atualizá-la" ) from e
    return radcom

In [ ]:
#export
def row2dict(row):
    """Receives a json row and return the dictionary from it"""
    #dict_result = {k: row[k] for k in cols}
    #dict_result['Número da Estação'] = row['administrativo']['@numero_estacao']
    return {k:v for k,v in row.items()}

def dict2cols(df, reject=()):
    """Recebe um dataframe com dicionários nas células e extrai os dicionários como colunas
       Opcionalmente ignora e exclue as colunas em reject
       """
    for column in df.columns:
        if column in reject:
            df.drop(column, axis=1, inplace=True)
            continue
        if type(df[column].iloc[0]) == collections.OrderedDict:
            try:
                new_df = pd.DataFrame(df[column].apply(row2dict).tolist())
                df = pd.concat([df, new_df], axis=1)
                df.drop(column, axis=1, inplace=True)
            except AttributeError:
                continue
    return df

def parse_plano_basico(row, cols=COL_PB):
    """Receives a json row and filter the column in `cols`"""
    return {k: row[k] for k in cols}

def read_estações(path):
    df = pdx.read_xml(path, ['estacao_rd'])
    df = pd.DataFrame(df['row'].apply(row2dict).tolist()).replace('', pd.NA)
    df = dict2cols(df)
    return df
    df = df.loc[:, COL_ESTACOES]
    df.columns = NEW_ESTACOES
    return df
    #df['Número da Estação'] = row['administrativo']['@numero_estacao']

def read_plano_basico(path):
    df = pdx.read_xml(path, ['plano_basico'])
    df = pd.DataFrame(df['row'].apply(row2dict).tolist()).replace('', pd.NA)
    df = dict2cols(df, REJECT_ESTACOES)
    df.loc[df['@Frequência'].isna(), '@Frequência'] = df.loc[df['@Frequência'].isna(), '@Frequencia']
    df.drop('@Frequencia', axis=1, inplace=True)
    df.loc[df['@País'].isna(), '@País'] = df.loc[df['@País'].isna(), '@Pais']
    df.drop('@Pais', axis=1, inplace=True)
    return df
    df = df.loc[:, COL_PB]
    df.columns = NEW_PB
    return df
#     df['@Longitude'] = df['@Longitude'].str.replace(',', '.').astype('float')
#     df['@Latitude'] = df['@Latitude'].str.replace(',', '.').astype('float')
#     df['@Frequência'] = df['@Frequência'].str.replace(',', '.').astype('float')


def read_historico(path):
    regex = r"\s([a-zA-Z]+)=\'{1}([\w\-\ :\.]*)\'{1}"
    with ZipFile(path) as xmlzip:
        with xmlzip.open('documento_historicos.xml', 'r') as xml:
            xml_list = xml.read().decode().split('\n')[2:-1]
    dict_list = []
    for item in xml_list:
        matches = re.finditer(regex, item, re.MULTILINE)
        dict_list.append(dict(match.groups() for match in matches))
    df = pd.DataFrame(dict_list)
    return df
    df = df[(df.tipoDocumento == 'Ato') & (df.razao == 'Autoriza o Uso de Radiofrequência')].reset_index()
    df = df.loc[:, ['id', 'numeroDocumento', 'orgao', 'dataDocumento']]
    df.columns = ['Id', 'Num_Ato', 'Órgao', 'Data_Ato']
    df['Data_Ato'] = pd.to_datetime(df.Data_Ato)
    return df.sort_values('Data_Ato').groupby('Id').last().reset_index()

In [ ]:
#export
def read_mosaico(pasta, update=False):
    if update:
        update_mosaico(pasta)
        estações = read_estações(f'{pasta}/estações.zip')
        plano_basico = read_plano_basico(f'{pasta}/plano_basico.zip')
        historico = read_historico(f'{pasta}/historico.zip')
        df = pd.merge(estações, plano_basico, on='Id').merge(historico, on='Id')
        df.to_feather(f'{pasta}/mosaico.fth')
        return df
    if not (file := Path(f'{pasta}/mosaico.fth')).exists():
            return read_mosaico(pasta, update=True)
    return pd.read_feather(f'{pasta}/mosaico.fth')

In [ ]:
df = read_mosaico(r'D:\OneDrive - ANATEL\GR01FI3\BaseDados', True)

Baixando as Estações do Mosaico...
Baixando o Plano Básico das Estações...
Baixando o Histórico de Atualizações...
Kbô


KeyError: 'Id'

In [ ]:
df

In [ ]:
h = read_historico(r'D:\OneDrive - ANATEL\GR01FI3\BaseDados\historico.zip')

In [ ]:
h

,item,id,entidade,cnpj,numeroProcesso,numeroDocumento,tipoDocumento,orgao,dataDocumento,dataDou,razao,natureza
0,1,57dbaad04f6cc,,,,,,,,,,
1,1,57dbaad04f6cc,,,,,,,,,,
2,1,57dbaad053c60,X-MEDIAGROUP S.A.,03211814000163,,0,Decreto,PR,2013-08-28 00:00:00.000,,Outorga,1
3,1,57dbaad053c60,X-MEDIAGROUP S.A.,03211814000163,,,,,,,,
4,1,57dbaad053c60,X-MEDIAGROUP S.A.,03211814000163,,123,Despacho,CMPRL,NaN,,Consol. Carac. Técnicas,Técnico
...,...,...,...,...,...,...,...,...,...,...,...,...
178853,81847,605b779008aac,,,,,,,,,,
178854,81847,605b8a0a2f75b,,,,,,,,,,
178855,81847,605b8a0a2f75b,,,,,,,,,,
178856,81847,606dca9d7e94a,,,,,,,,,,


In [ ]:
p = read_plano_basico(r'D:\OneDrive - ANATEL\GR01FI3\BaseDados\plano_basico.zip')

In [ ]:
p.columns

Index(['@item', '@id', '@Municipio', '@Classe', '@Servico', '@Entidade',
       '@LatitudeGMS', '@LongitudeGMS', '@Latitude', '@Longitude', '@ERP_Dia',
       '@NumTorres_Diurno', '@Altura_Diurno', '@K_Diurno', '@Psi_Diurno',
       '@Espaçamento_Diurno', '@Azimute_Diurno', '@ERP_Noturno',
       '@NumTorres_Noturno', '@Altura_Noturno', '@K_Noturno', '@Psi_Noturno',
       '@Espacamento_Noturno', '@Azimute_Noturno', '@CampoCaracteristico',
       '@País', '@Canal', '@Frequência', '@Decalagem', '@ERP', '@Altura',
       '@IdtPlanoBasico', '@UF', '@CodMunicipio', '@Status', '@CNPJ',
       '@Observacoes', '@Fistel', '@Carater', '@Limitacoes'],
      dtype='object')

In [ ]:
e = read_estações(r'D:\OneDrive - ANATEL\GR01FI3\BaseDados\estações.zip')

In [ ]:
e

,@item,@SiglaServico,@id,@state,@entidade,@fistel,@cnpj,@CodMunicipio,@municipio,@uf,...,@antena_Auxiliar_Fabricante,@antena_Auxiliar_Modelo,@antena_Auxiliar_Ganho_dBd,@antena_Auxiliar_BeamTilt,@antena_Auxiliar_OrientacaoNV,@antena_Auxiliar_Polarizacao,@antena_Auxiliar_HCI,@antena_Auxiliar_Nulos,@antena_Auxiliar_Observacao,item
0,1,TV,57dbaad053c60,TV-C1,X-MEDIAGROUP S.A.,50410887137,03211814000163,1200336,Mâncio Lima,AC,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
1,2,TV,57dbaad056910,TV-CE-01,FUNDACAO ALDEIA DE COMUNICACAO DO ACRE-FUNDAC,50011562544,15254447000164,1200401,Rio Branco,AC,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
2,3,TV,57dbaad05ae4d,TV-CE-01,RADIO TV DO AMAZONAS LTDA,12008000443,04387825000161,1200401,Rio Branco,AC,...,,,,,,,,,,"{'@seq': '1', '@dia_inicio': 'dom', '@dia_fim'..."
3,4,TV,57dbaad060a1e,TV-CE-01,SOCIEDADE ACREANA DE COMUNICACAO FRONTEIRA LTDA,17000000201,14366199000180,1200401,Rio Branco,AC,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
4,5,TV,57dbaad064c5c,TV-CE-01,RADIO E TELEVISAO NORTE LTDA,17000001100,14339220000159,1200401,Rio Branco,AC,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28048,28049,RTVD,5f32c1c918e6b,TV-C2,FUNDACAO EDUCACIONAL E CULTURAL DE IPANEMA,50433856696,04608796000110,3156700,Sabará,MG,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
28049,28050,FM,5f36a03433642,FM-C2,EMPRESA PAULISTA DE RADIODIFUSAO LTDA,50437944611,44872109000194,3542404,Regente Feijó,SP,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
28050,28051,FM,5f68d432841a5,FM-C2,RADIO ITAPIRANGA LTDA,50433860456,84375872000124,4208401,Itapiranga,SC,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."
28051,28052,FM,5f8dcc96f23f9,FM-C1,EMISSORAS SUL BRASILEIRAS LTDA,50433937009,95818506000119,4309605,Horizontina,RS,...,,,,,,,,,,"{'@seq': '0', '@dia_inicio': '', '@dia_fim': '..."


In [ ]:
[c for c in e.columns]

['@item',
 '@SiglaServico',
 '@id',
 '@state',
 '@entidade',
 '@fistel',
 '@cnpj',
 '@CodMunicipio',
 '@municipio',
 '@uf',
 'enderecos',
 'estacao_auxiliar',
 'rds',
 '@entidade_nome_entidade',
 '@entidade_nome_fantasia',
 '@entidade_ddd',
 '@entidade_telefone',
 '@entidade_email',
 '@entidade_codtipotaxa',
 '@entidade_tipotaxa',
 '@entidade_idttipoorgao',
 '@entidade_tipoorgao',
 '@localEspecifico',
 '@responsavel_legal_cpf',
 '@responsavel_legal_nome_completo',
 '@responsavel_legal_email',
 '@num_servico',
 '@srd_planobasico_ind_carater',
 '@num_fistel',
 '@habilitacao_NumScradJur',
 '@habilitacao_NumScradTec',
 '@habilitacao_DataLimiteInstalacao',
 '@habilitacao_DataValFreq',
 '@habilitacao_DataContrato',
 '@habilitacao_IndEducativo',
 '@network',
 '@carater',
 '@fase',
 '@canalCidadania',
 '@codFinalidade',
 '@finalidade',
 'observacao',
 '@habilitacao_NumFistelGeradora',
 '@numero_estacao',
 '@NomeIndicativo',
 '@FindCodSituacaoLicenca',
 '@FindDatalimiteInstalacao',
 '@DataEmiss

In [ ]:
e['@NumLicenca'].unique()

array(['', '53500.022699/2018-21', '003937/2003', ...,
       '53500.035492/2020-31', '53500.005414/2021-93',
       '53500.019894/2021-70'], dtype=object)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted eda.ipynb.
Converted filter.ipynb.
Converted index.ipynb.
Converted parser.ipynb.
Converted queries.ipynb.
Converted tests.ipynb.
